In [ ]:
import sys, platform

base_path = '/Users/yuri/git/diasYuri/tcc/radiation-forecast-mg'

if platform.system() is 'Darwin':
    base_path = '/Users/yuri/git/diasYuri/tcc/radiation-forecast-mg'
sys.path.insert(0, base_path)

In [ ]:
from math import sqrt

import keras
import numpy as np
import pandas as pd
import optuna
from keras import layers
from keras.callbacks import EarlyStopping
from keras_tuner.tuners import RandomSearch
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM, InputLayer
from keras.callbacks import EarlyStopping
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from pandas import DataFrame

import src.filler as fl
from src.filler import RandomForestFillerWithOneModel, LstmFillerModel, HodmdFiller
from src.dataframe import read_data_from_csv

In [ ]:
filename = f'{base_path}/data/daily_vicosa.csv'

In [ ]:
df = read_data_from_csv(filename)

df = df.RADIATION

In [ ]:
data = fl.FillerHelper.get_largest_complete_interval(df)
data_with_gaps = fl.FillerHelper.introduce_gaps(data.copy(), 
                                                missing_percentage=0.02, 
                                                min_gap_size=1, 
                                                max_gap_size=20)

In [ ]:
maxrange = 1200

print(len(data))
print(data_with_gaps.isna().sum())
print('Percentual de dados faltantes:', (data_with_gaps.isna().sum()/len(data)))

pyplot.figure(figsize=(12, 6))
pyplot.plot(data[0:maxrange], color = 'b', linestyle = ':')
pyplot.plot(data_with_gaps[0:maxrange], color='green', label='missing_data')
pyplot.show()

DMD

In [ ]:
def objective(trial):
    dmd_data = data_with_gaps.copy()
    filler = fl.HodmdFiller(d_factor=trial.suggest_float("d_fator", 0.5, 0.99, log=True))
    data_filled = filler.dmd_filler(dmd_data)
    mape = mean_absolute_error(data, data_filled)
    print('MAPE SCORE:', mape)
    return mape


study = optuna.create_study(
    direction='minimize',
    storage='sqlite:///filler_method.db',
    study_name=f'hodmd',
    load_if_exists=True
)

n_trials = 16
if len(study.trials) < n_trials:
    study.optimize(
        objective, 
        n_trials=n_trials, 
        timeout=600, 
        gc_after_trial=True,
        n_jobs=4
    )   

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Interpolation Seasonal

In [ ]:
def objective(trial):
    data_filled = fl.seasonal_filler(data_with_gaps.copy(), 
                                     period=trial.suggest_int("period", 24, 365, log=True), 
                                     factor=trial.suggest_int("factor", 1, 6, log=True))
    mae = mean_absolute_error(data, data_filled)
    print('MAE SCORE:', mae)
    print('Faltantes SCORE:', data_filled.isna().sum())
    return mae



study = optuna.create_study(
    direction='minimize',
    storage='sqlite:///filler_method.db',
    study_name=f'interpolation_seasonal',
    load_if_exists=True
)

n_trials = 160
if len(study.trials) < n_trials:
    study.optimize(
        objective, 
        n_trials=n_trials, 
        timeout=600, 
        gc_after_trial=True,
        n_jobs=4
    )   

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))